In [2]:
import pickle
import pandas as pd
import numpy as np

import re

from sklearn.feature_extraction.text import CountVectorizer


import matplotlib.pyplot as plt

from sklearn.preprocessing import Normalizer, StandardScaler

from tqdm import tqdm

import spacy
import requests
from scipy.spatial import distance

import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
import pprint
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

# Befor start - Spacy configuration
* optional step. model is saved in the file, but if you want to run it by your own, then you need to instal and configure Spacy on your machine

In [3]:
# przygotowanie środowiska, komendy linux
# instalacja morfeusza2
# zrobić w terminalu

# wget -O - http://download.sgjp.pl/apt/sgjp.gpg.key|sudo apt-key add -
# sudo apt-add-repository http://download.sgjp.pl/apt/ubuntu
# sudo apt update
# sudo apt install morfeusz2
# sudo apt install python3-morfeusz2

# instalacja spacy
# python3 -m pip install spacy

# pobieranie morfeusza
# http://morfeusz.sgjp.pl/download/

# spacy z ipi pan
# https://github.com/ipipan/spacy-pl


# instalacja modelu ipi pan dla języka polskiego - trudniejsze, nie działa aktualnie
# wget "http://zil.ipipan.waw.pl/SpacyPL?action=AttachFile&do=get&target=pl_spacy_model_morfeusz-0.1.3.tar.gz"
# python -m pip install pl_spacy_model_morfeusz-0.1.3.tar.gz


# problem z no module found
# https://towardsdatascience.com/importerror-no-module-named-xyz-45e4a5339e1b


# instalacja oficjalnego modelu spacy
# python -m spacy download pl_core_news_sm


# dodatkowe zależności:
# !python3 -m pip install tqdm
# !python3 -m pip install networkx

# linkowanie modelu do spacy (to zadziałało - możliwe, że pomogło też zainstalowanie spacy na condzie)
# python3 -m spacy link pl_core_news_sm pl_core_news_sm -f

# instalacja większego mdoelu
# !python3 -m spacy download pl_core_news_lg
# !python3 -m spacy link pl_core_news_lg pl_core_news_lg -f

First, we need to load pretrained statistical models for Polish

In [8]:
nlp = spacy.load("pl_core_news_lg")

Next we load our preprocessed data

In [4]:
df_working = pd.read_pickle("data/df_working_pickle.pkl")
df_working

,link,recipe_name,meal_type_detailed,ingredients,instructions,meal_type,raw_ingredients,cleaned_ingredients,ingredients_count,cusine_style
0,/ravioli-z-cukinia-i-ricotta,Ravioli z cukinią i ricottą,Bez mięsa,"cukinia – 1 szt., oliwa z oliwek – 1 łyżka, cz...",Przygotuj: tarkę o grubych i drobnych oczkach...,glowne,"cukinia – 1 szt., oliwa z oliwek – 1 łyżka, cz...","cukinia, oliwa z oliwek, czosnek, świeża bazyl...",22,wloska
1,/galettes-bretonnes-czyli-nalesniki-po-bretonsku,"Galettes bretonnes, czyli naleśniki po bretońsku",Naleśniki na słono,"mąka pszenna – 200 g, mąka gryczana – 100 g, m...",Przygotuj: patelnię do naleśnik&#243;w tarkę ...,glowne,"mąka pszenna – 200 g, mąka gryczana – 100 g, m...","mąka pszenna, mąka gryczana, mleko, jajka, wod...",12,francuska
2,/galettes-bretonnes-czyli-nalesniki-po-bretonsku,"Galettes bretonnes, czyli naleśniki po bretońsku",Naleśniki na słono,"mąka pszenna – 200 g, mąka gryczana – 100 g, m...",Przygotuj: patelnię do naleśnik&#243;w tarkę ...,sniadania,"mąka pszenna – 200 g, mąka gryczana – 100 g, m...","mąka pszenna, mąka gryczana, mleko, jajka, wod...",12,francuska
3,/kluski-dyniowe-z-jogurtem-i-owocami,Kluski dyniowe z jogurtem i owocami,Kluski,"dynia piżmowa – 1 szt., purée z dyni – 500 g, ...",Przygotuj: piekarnik rozgrzany do temperatury...,glowne,"dynia piżmowa – 1 szt., purée z dyni – 500 g, ...","dynia piżmowa, purée z dyni, mąka pszenna, mąk...",17,polska
4,/kurczak-w-sosie-jogurtowym,Kurczak w sosie jogurtowym,Kurczak,"filet z piersi kurczaka – 400 g, oliwa z oliwe...",Przygotuj: tarkę o drobnych oczkach ręcznik p...,glowne,"filet z piersi kurczaka – 400 g, oliwa z oliwe...","filet z piersi kurczaka, oliwa z oliwek, masło...",16,NaN
...,...,...,...,...,...,...,...,...,...,...
2785,/mus-czekoladowy-z-malinami,Mus czekoladowy z malinami,Desery czekoladowe,"100 g świeżych lub mrożonych malin, 15 g cukru...",Przygotowanie Żelatynę moczymy w zimnej wodzi...,desery,"100 g świeżych lub mrożonych malin, 15 g cukru...","g świeżych lub mrożonych malin, g cukru pudr...",8,NaN
2786,/jablka-zapiekane-z-zurawina,Jabłka zapiekane z żurawiną,Desery z owocami,"50 g białek jaj (2 białka jaj), 70 g cukru, 30...",Masa bezowa Białka ubijamy na sztywną masę. D...,desery,"50 g białek jaj (2 białka jaj), 70 g cukru, 30...","g białek jaj ( białka jaj), g cukru, g migd...",5,NaN
2787,/panna-cotta-waniliowa-z-owocami-lasu,Panna cotta waniliowa z owocami lasu,Desery z owocami,"250 ml śmietanki 30%, 250 ml mleka 3,2%, 80-10...","Przygotowanie Mleko, śmietankę i cukier podgr...",desery,"250 ml śmietanki 30%, 250 ml mleka 3,2%, 80-10...","ml śmietanki , ml mleka , g cukru kryształu...",11,wloska
2788,/letnie-owoce-pod-kruszonka,Letnie owoce pod kruszonką,Desery z owocami,"płatki owsiane Nordwaldtaler: 0,75 szklanki, m...",Przygotowanie Płatki i ziarno słonecznika pra...,desery,"płatki owsiane Nordwaldtaler: 0,75 szklanki, m...","płatki owsiane Nordwaldtaler, masło Pilos do k...",12,NaN


We will try a few approaches to find the best way to get alternatives for the ingredients used in a chosen recipe.
1. spacy cosine similarity based on prelearned model
2. nltk similarity based on data from kuchnia lidla
3. classification algorithms

# 1. Spacy similarity

First I define the function what will choose only nouns and adjectives from the list of ingredients. It will help getting the same form of the same words in different forms ('mąki' vs. 'mąka') and number ('jajko' vs. 'jajka'). It will also clean punctuation and other unnecessary words like verbs. Unfortunately, this approach is still not perfect, because we will get words like 'tabliczka'. It will also return unnecessary adjectives like 'dodatkowy' or  'dodatkowy', but adjectives might be useful for words like 'pszenny' in regards to type of flour.

In [9]:
def lemantize(text):
    stoplist=['1', '½', 'łyżka', 'łyżeczka', 'pęczek', 'szczypta', 'szklanka', 'garść', 'sztuka', '%', '1/8','-0,5',
 '1/10', '1/2',  '1/3', '1/4', '10', '11/2', '150ml', '1:1', '1l', '1¼', '1½', '1¾', '1⅓',
 '2', '2:1', '2l', '2¼', '2¾', '3', '4', '5', '7x7', 'Celinka', 'Karol', 'Kinga', 'Małecki', 'Mistera', 'a',]
    lemats_list=[]
    for word in nlp(text):
        if word.pos_ == 'NOUN' or word.pos_ == 'ADJ':
            if word.lemma_ not in stoplist:
                lemats_list.append(word.lemma_)
    lemats_list = (', ').join(set(lemats_list))
    return lemats_list

def lemantize_nouns(text):
    stoplist=['1', '½', 'łyżka', 'łyżeczka', 'pęczek', 'szczypta', 'szklanka', 'garść', 'sztuka', '%', '1/8','-0,5',
 '1/10', '1/2',  '1/3', '1/4', '10', '11/2', '150ml', '1:1', '1l', '1¼', '1½', '1¾', '1⅓',
 '2', '2:1', '2l', '2¼', '2¾', '3', '4', '5', '7x7', 'Celinka', 'Karol', 'Kinga', 'Małecki', 'Mistera', 'a',]
    lemats_list=[]
    for word in nlp(text):
        if word.pos_ == 'NOUN':
            if word.lemma_ not in stoplist:
                lemats_list.append(word.lemma_)
    lemats_list = (', ').join(set(lemats_list))
    return lemats_list

In [10]:
%%time

df_working['lematized_ingredients'] = df_working.apply(lambda x: lemantize(x['cleaned_ingredients']),axis=1)
df_working['lematized_ingredients_nouns'] = df_working.apply(lambda x: lemantize_nouns(x['cleaned_ingredients']),axis=1)


CPU times: user 1min 9s, sys: 0 ns, total: 1min 9s
Wall time: 1min 9s


How the list of lematized ingredient looks?

In [12]:
df_working.iloc[0]['lematized_ingredients']

'jajko, semolina, oliwka, bazylia, sól, chili, żółtko, cukinia, ciepły, czosnek, oliwa, pestka, woda, czarny, parmezan, ser, ricotta, tortowy, mielony, olej, masło, winogrono, mąka, świeży, pieprz'

In [13]:
df_working.iloc[0]['lematized_ingredients_nouns']

'jajko, semolina, bazylia, oliwka, sól, chili, żółtko, cukinia, czosnek, oliwa, pestka, woda, parmezan, ser, ricotta, olej, masło, winogrono, mąka, pieprz'

In general, it looks promising. With more time, stoplist could be updated even more, but majority of ingredients looks fine. Now, we need to get unique list of words for count vectoriser. It is needed for 2 reasons:
1. Dense matrix will be used for classification. Each ingredient will be separate feature for finding alternatives via classifiers.
2. Additionally, it will be used in the function to find the recipe based on the ingredients that the user will pass to the program.

In [10]:
# wyciągamy słowa z listy składników
words = df_working['lematized_ingredients'].tolist()
# łączymy je w listę żeby wyciągnąć z niej unikalne słowa
unique_words = set((', ').join(words).split(', '))


cv = CountVectorizer(vocabulary=unique_words)
df_ingredients_vectorized = cv.fit_transform(df_working['lematized_ingredients'])
df_ingredients_vectorized = pd.DataFrame(df_ingredients_vectorized.todense(), columns=unique_words)

# no to ile jest tych unikalnych słów?
len_of_unique_words = len(unique_words)
print(f'There are {len_of_unique_words} unique words in ingredients')

There are 1756 unique words in ingredients


In [11]:
df_ingredients_vectorized.columns

Index(['tarka', 'sweet', 'mirin', 'ząbek', 'biopomarańcza', 'basmati',
       'botwinka', 'garam', 'nashi', 'otarta',
       ...
       'kotlet', 'orzechowy', 'gochujang', 'centymetrowy', 'daga', 'mostek',
       'koci', 'wyszorowana', 'tortownica', 'krótki'],
      dtype='object', length=1756)

In [12]:
%%time
def check_similarity(list_of_ingredients):
    similarity_dict_spacy={}
    for i in list_of_ingredients:
        similarity_dict_spacy[i]=[nlp(i).similarity(nlp(j)) for j in list_of_ingredients]
    return similarity_dict_spacy

# WARNING! FUNCTION BELOW WILL START THE PROCESS OF PREPARING SIMILARITIES MATRIX, DO NOT UNCOMMENT IT 
# testing_similarities = check_similarity(df_ingredients_vectorized.columns)


CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 10.5 µs


In [13]:
"""
CPU times: user 12h 1min 3s, sys: 2.76 s, total: 12h 1min 5s
Wall time: 12h 1min 3s
"""

'\nCPU times: user 12h 1min 3s, sys: 2.76 s, total: 12h 1min 5s\nWall time: 12h 1min 3s\n'

In [14]:
# ingredients_similarity_matrix = pd.DataFrame(testing_similarities)
# ingredients_similarity_matrix['index'] = df_ingredients_vectorized.columns
# ingredients_similarity_matrix.set_index('index', inplace=True)

# !WARNING! DON'T UNCOMMENT THE LINE BELOW
# ingredients_similarity_matrix.to_pickle("ingredients_similarity_matrix.pkl")



In [15]:
ingredients_similarity_matrix = pd.read_pickle("ingredients_similarity_matrix.pkl")

In [16]:
# %%time
# similarity_dict_spacy={}
# for i in df_ingredients_vectorized.columns:
#     token1=nlp(i)
#     token2=nlp('kurczak')
#     similarity_dict_spacy[i]=token1.similarity(token2)

In [17]:
# df_similarity_spacy = pd.DataFrame.from_dict(similarity_dict_spacy, orient='index')
# df_similarity_spacy = df_similarity_spacy.sort_values(by=[0], ascending=False)
# df_similarity_spacy.head(10)

In [18]:
ingredients_input = 'olej, czosnek, oliwka, bazylia, sól, pestka, oliwa, ciepły, kurczak'

In [19]:
def find_missing_ingredients(ingredients_input):
    if type(ingredients_input) == str:
        ingredients_input = ingredients_input.split(", ")
    missing_ingredients = []
    # check if all products are in similarity matrix
    for ingredient in ingredients_input:
        if ingredient not in ingredients_similarity_matrix:
    #         add columns and rows so after getting similarity data they are the same lenght as colm and row
            ingredients_similarity_matrix[ingredient] = 0
#             ingredients_similarity_matrix.loc[len(ingredients_similarity_matrix.index)]=0
#             ingredients_similarity_matrix.append(pd.Series(0, index=ingredients_similarity_matrix.columns))
            ingredients_similarity_matrix.loc[ingredient] = pd.Series(0)
            missing_ingredients.append(ingredient)

            ingredient_similarity = [nlp(ingredient).similarity(nlp(i)) for i in ingredients_similarity_matrix.columns]
    #         adding column
            ingredients_similarity_matrix[ingredient] = ingredient_similarity
    #         adding row
            ingredients_similarity_matrix.loc[ingredient] = ingredient_similarity
        else:
            pass
    print(missing_ingredients)

In [20]:
find_missing_ingredients(ingredients_input)

[]


In [21]:
# ingredients_similarity_matrix

In [22]:
from itertools import combinations

def make_ingredient_list_variants(ingredients_input, min_number_of_ingredient=5):
    all_ingredient_combinations = []
    
    if type(ingredients_input) == str:
        ingredients_input = ingredients_input.split(", ")

    for r in range(len(ingredients_input)+1):
        combinations_list = list(combinations(ingredients_input, r))
#         all_ingredient_combinations.append(list(combinations_list))
        
#         all_ingredient_combinations=[combinations_list for i in combinations_list if len(i)>min_number_of_ingredient]

        for i in combinations_list:
            if len(i)>min_number_of_ingredient:
                all_ingredient_combinations.append(list(i))
    return all_ingredient_combinations



In [23]:
make_ingredient_list_variants(ingredients_input)

[['olej', 'czosnek', 'oliwka', 'bazylia', 'sól', 'pestka'],
 ['olej', 'czosnek', 'oliwka', 'bazylia', 'sól', 'oliwa'],
 ['olej', 'czosnek', 'oliwka', 'bazylia', 'sól', 'ciepły'],
 ['olej', 'czosnek', 'oliwka', 'bazylia', 'sól', 'kurczak'],
 ['olej', 'czosnek', 'oliwka', 'bazylia', 'pestka', 'oliwa'],
 ['olej', 'czosnek', 'oliwka', 'bazylia', 'pestka', 'ciepły'],
 ['olej', 'czosnek', 'oliwka', 'bazylia', 'pestka', 'kurczak'],
 ['olej', 'czosnek', 'oliwka', 'bazylia', 'oliwa', 'ciepły'],
 ['olej', 'czosnek', 'oliwka', 'bazylia', 'oliwa', 'kurczak'],
 ['olej', 'czosnek', 'oliwka', 'bazylia', 'ciepły', 'kurczak'],
 ['olej', 'czosnek', 'oliwka', 'sól', 'pestka', 'oliwa'],
 ['olej', 'czosnek', 'oliwka', 'sól', 'pestka', 'ciepły'],
 ['olej', 'czosnek', 'oliwka', 'sól', 'pestka', 'kurczak'],
 ['olej', 'czosnek', 'oliwka', 'sól', 'oliwa', 'ciepły'],
 ['olej', 'czosnek', 'oliwka', 'sól', 'oliwa', 'kurczak'],
 ['olej', 'czosnek', 'oliwka', 'sól', 'ciepły', 'kurczak'],
 ['olej', 'czosnek', 'oliwka

In [24]:
# def get_ingredients_alternatives(ingredients_input, similarity_threshold=0.5):
#     alternatives_dict = {}
#     if type(ingredients_input) == str:
#         ingredients_input = ingredients_input.split(", ")
#     for ingredient in ingredients_input:
# #         ranking = ingredients_similarity_matrix \
# #                                     .sort_values(by=[ingredient], ascending=False)[ingredient] \
# #                                     .head(3) \
# #                                     .reset_index()
#         ranking = ingredients_similarity_matrix[ingredient][(ingredients_similarity_matrix[ingredient]!=1) & \
#                                           (ingredients_similarity_matrix[ingredient]>similarity_threshold)] \
#                                         .sort_values(ascending=False) \
#                                         .reset_index()
#         alternatives_dict[ingredient] = list(zip(ranking['index'].tolist(), ranking[ingredient].tolist()))

#     return alternatives_dict

def get_ingredients_alternatives(ingredients_input, similarity_threshold=0.8):
    alternatives_dict = {}
    
    if type(ingredients_input) == str:
        ingredients_input = ingredients_input.split(", ")

    for ingredient in ingredients_input:
        ranking = ingredients_similarity_matrix[ingredient][(ingredients_similarity_matrix[ingredient]!=1) & \
                                          (ingredients_similarity_matrix[ingredient]>similarity_threshold)] \
                                        .sort_values(ascending=False)
        alternatives_dict[ingredient] = ranking.index.tolist()

    return alternatives_dict

In [25]:
# ingredients_input = ['jabłko', 'jajka', 'mąka', 'szynka', 'pomidor']
get_ingredients_alternatives(ingredients_input, similarity_threshold=0.7)


{'olej': ['olejek'],
 'czosnek': ['imbir', 'majeranek'],
 'oliwka': [],
 'bazylia': [],
 'sól': [],
 'pestka': [],
 'oliwa': [],
 'ciepły': ['chłodny', 'zimny', 'gorący'],
 'kurczak': ['indyk', 'makaron']}

In [26]:
from random import sample 
def combine_ingr_and_alt_into_combinations(ingredients_input, similarity_threshold=0.7):
    if type(ingredients_input) == str:
        ingredients_input = ingredients_input.split(", ")
        
    alternatives = [*get_ingredients_alternatives(ingredients_input, similarity_threshold=similarity_threshold).values()]
    alternatives = [item for sublist in alternatives for item in sublist]
    merged_list = alternatives + ingredients_input
    combinations_of_merged_list = make_ingredient_list_variants(set(merged_list))
    combinations_of_merged_list_len = len(combinations_of_merged_list)
    if combinations_of_merged_list_len>10000:
        return sample(combinations_of_merged_list, 10000)
    else:
        return combinations_of_merged_list




In [ ]:
# combine_ingr_and_alt_into_combinations(ingredients_input, similarity_threshold=0.8)

# 2. NLTK

In [11]:
%%time
import nltk
from nltk import word_tokenize
from nltk.util import ngrams
from collections import Counter
import gensim

instructions_list_merged = []

for _, row in df_working.iterrows():
    instructions = row.lematized_ingredients_nouns
    instructions_list = lematized_ingredients_nouns.split()
    instructions_list_merged.append(instructions_list)


# https://www.kaggle.com/alvations/word2vec-embedding-using-gensim-and-nltk
from gensim.models import Word2Vec


model_instructions = gensim.models.Word2Vec(instructions_list_merged)

NameError: name 'lematized_ingredients_nouns' is not defined

In [ ]:
model_instructions.most_similar(positive=['cukinia'], topn = 10)

In [14]:
%%time
# df_working['lematized_instructions'] = df_working.apply(lambda x: lemantize_sentences(x['instructions']),axis=1)

instructions_lematized_list_merged = []

for _, row in df_working.iterrows():
    lematized_instructions = row.lematized_ingredients_nouns
    lematized_instructions_list = lematized_instructions.split()
    instructions_lematized_list_merged.append(lematized_instructions_list)


# https://www.kaggle.com/alvations/word2vec-embedding-using-gensim-and-nltk
from gensim.models import Word2Vec


model_lematized_instructions = gensim.models.Word2Vec(instructions_lematized_list_merged)




CPU times: user 821 ms, sys: 16 ms, total: 837 ms
Wall time: 674 ms


In [19]:
model_lematized_instructions.most_similar(positive=['czekolada'], topn = 10)

/home/d0lar/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('żelatyna', 0.9996838569641113),
 ('budyń,', 0.9996405243873596),
 ('migdał', 0.9996094703674316),
 ('typ,', 0.9995778799057007),
 ('żelatyna,', 0.9995228052139282),
 ('wiórek,', 0.9994864463806152),
 ('banan,', 0.9994816780090332),
 ('czekolada,', 0.999409556388855),
 ('cynamon,', 0.9994093775749207),
 ('posypania,', 0.9994043707847595)]

## 2.2 ngrams

In [ ]:
import nltk
from nltk import word_tokenize
from nltk.util import ngrams
from collections import Counter

redundant_tokens=[',','z','%','g','l']
sentences=(" ").join(df_working['lematized_ingredients'].tolist())
sentences = (" ").join(sentences.split(', '))


# filtered_text1 = [w for w in sentences]
# sentences_merged = ". ".join(sentences)

token = nltk.word_tokenize(sentences)


bigrams = ngrams(token,2)
trigrams = ngrams(token,3)
fourgrams = ngrams(token,4)
fivegrams = ngrams(token,5)

In [ ]:

print(Counter(trigrams))
# df_working['lematized_ingredients'].tolist()

In [ ]:
df_working

# 3. Classifier

In [27]:
# https://www.mikulskibartosz.name/how-to-split-a-list-inside-a-dataframe-cell-into-rows-in-pandas/
missing_ingredient_list = []
meal_type_list = []
link_list = []
rest_of_ingredients_list = []
meal_type_detailed_list = []
raw_ingredients_list = []
lematized_ingredients_list = []

for _, row in df_working.iterrows():
    meal_type = row.meal_type
    link = row.link
    meal_type_detailed = row.meal_type_detailed
    raw_ingredients = row.ingredients
    lematized_ingredients = row.lematized_ingredients
    for ingredient in row.lematized_ingredients.split(", "):
        meal_type_list.append(meal_type)
        missing_ingredient_list.append(ingredient)
        link_list.append(link)
        rest_of_ingredients_list.append(", ".join([x for x in row.lematized_ingredients.split(", ") \
                                                   if x!=ingredient]))
        meal_type_detailed_list.append(meal_type_detailed)
        raw_ingredients_list.append(raw_ingredients)
        lematized_ingredients_list.append(lematized_ingredients)

df_missing_ingr_clf = pd.DataFrame({
    "link": link_list,
    "lematized_ingredients":lematized_ingredients_list,
    "missing_ingredient": missing_ingredient_list,
    "meal_type_list": meal_type_list,
    "rest_of_ingredients": rest_of_ingredients_list,
    "meal_type_detailed": meal_type_detailed_list,
    "raw_ingredients_list":raw_ingredients_list,
})

df_missing_ingr_clf

,link,lematized_ingredients,missing_ingredient,meal_type_list,rest_of_ingredients,meal_type_detailed,raw_ingredients_list
0,/ravioli-z-cukinia-i-ricotta,"ciepły, olej, chili, mąka, żółtko, oliwka, sem...",ciepły,glowne,"olej, chili, mąka, żółtko, oliwka, semolina, m...",Bez mięsa,"cukinia – 1 szt., oliwa z oliwek – 1 łyżka, cz..."
1,/ravioli-z-cukinia-i-ricotta,"ciepły, olej, chili, mąka, żółtko, oliwka, sem...",olej,glowne,"ciepły, chili, mąka, żółtko, oliwka, semolina,...",Bez mięsa,"cukinia – 1 szt., oliwa z oliwek – 1 łyżka, cz..."
2,/ravioli-z-cukinia-i-ricotta,"ciepły, olej, chili, mąka, żółtko, oliwka, sem...",chili,glowne,"ciepły, olej, mąka, żółtko, oliwka, semolina, ...",Bez mięsa,"cukinia – 1 szt., oliwa z oliwek – 1 łyżka, cz..."
3,/ravioli-z-cukinia-i-ricotta,"ciepły, olej, chili, mąka, żółtko, oliwka, sem...",mąka,glowne,"ciepły, olej, chili, żółtko, oliwka, semolina,...",Bez mięsa,"cukinia – 1 szt., oliwa z oliwek – 1 łyżka, cz..."
4,/ravioli-z-cukinia-i-ricotta,"ciepły, olej, chili, mąka, żółtko, oliwka, sem...",żółtko,glowne,"ciepły, olej, chili, mąka, oliwka, semolina, m...",Bez mięsa,"cukinia – 1 szt., oliwa z oliwek – 1 łyżka, cz..."
...,...,...,...,...,...,...,...
55900,/mus-czekoladowy,"fin, śmietanka, carre, mleczny, czekolada, pil...",carre,desery,"fin, śmietanka, mleczny, czekolada, pilos, gorzki",Desery czekoladowe,"śmietanka Pilos 30%, mocno schłodzona: 300 ml,..."
55901,/mus-czekoladowy,"fin, śmietanka, carre, mleczny, czekolada, pil...",mleczny,desery,"fin, śmietanka, carre, czekolada, pilos, gorzki",Desery czekoladowe,"śmietanka Pilos 30%, mocno schłodzona: 300 ml,..."
55902,/mus-czekoladowy,"fin, śmietanka, carre, mleczny, czekolada, pil...",czekolada,desery,"fin, śmietanka, carre, mleczny, pilos, gorzki",Desery czekoladowe,"śmietanka Pilos 30%, mocno schłodzona: 300 ml,..."
55903,/mus-czekoladowy,"fin, śmietanka, carre, mleczny, czekolada, pil...",pilos,desery,"fin, śmietanka, carre, mleczny, czekolada, gorzki",Desery czekoladowe,"śmietanka Pilos 30%, mocno schłodzona: 300 ml,..."


In [30]:
corpus = df_missing_ingr_clf['lematized_ingredients'].tolist()

unique_words = set((', ').join(corpus).split(', '))

cv = CountVectorizer(analyzer='word', tokenizer=lambda x: x.split(", "))
df_ingredients_vectorized_clf = cv.fit_transform(df_missing_ingr_clf['lematized_ingredients'])
# print(vectorizer.get_feature_names())

# print(ingr_vectors.toarray())
df_ingredients_vectorized_clf = pd.DataFrame(df_ingredients_vectorized_clf.todense(), columns=cv.get_feature_names())


len_of_unique_words = len(unique_words)
print(f'Mamy {len_of_unique_words} unikalnych słów w korpusie')

Mamy 1756 unikalnych słów w korpusie


## model train

In [33]:
X_train, X_test, y_train, y_test = train_test_split(df_missing_ingr_clf['lematized_ingredients'], 
                                                    df_missing_ingr_clf['link'], 
                                                    test_size=0.1, 
                                                    random_state=42)

In [34]:
%%time
pipe = Pipeline([("vectorizer", CountVectorizer(analyzer='word', tokenizer=lambda x: x.split(", "))),
                 ("regresja", LogisticRegression(solver="liblinear"))])
pipe.fit(df_missing_ingr_clf['lematized_ingredients'], df_missing_ingr_clf['link'])

accuracy_score(pipe.predict(X_test), y_test)

# Model need to be run, because can't pickle lambda funciton
# https://stackoverflow.com/questions/25348532/can-python-pickle-lambda-functions/25353243#25353243

CPU times: user 11min 19s, sys: 190 ms, total: 11min 19s
Wall time: 11min 19s


0.9998211411196566

In [35]:
X_test

38284    świeży, śmietankowy, mięta, serek, truskawka, ...
33788    rukola, pełnoziarnistego, chleb, musztarda, op...
2008     świeży, wytrawny, winogrono, pieprz, malinowy,...
22071    bazylia, sól, wędzony, szalotka, dojrzały, sur...
27166    żelującego, smak, woda, mięta, truskawka, sok,...
                               ...                        
55631    smak, śmietanka, wiśniowy, żelatyna, agrestowy...
33184    masło, sól, mąka, wiśnia, jajko, cynamon, słoi...
5195     cukinia, kukurydziany, czosnek, drożdże, suchy...
29472    musztarda, świeży, ocet, cebula, jaglany, piep...
36043    posiekanym, świeży, jogurt, ostry, żółty, piep...
Name: lematized_ingredients, Length: 5591, dtype: object

In [ ]:
# https://stackoverflow.com/questions/38403951/sklearn-how-to-predict-probability-for-all-target-labels
# predictions = pipe.predict_proba(['olej, czosnek, oliwka, bazylia, sól, pestka, oliwa, ciepły, semolina, czarny, mielony, masło, parmezan, żółtko, świeży, jajko, mąka, pieprz, tortowy, winogrono, ser, chili, ricotta, woda, cukinia'])

# list_of_predictions = list(zip(list(pipe.classes_), predictions[0,:]))
# list_of_predictions = pd.DataFrame(list_of_predictions)
# list_of_predictions = list_of_predictions.sort_values(by=[1], ascending=False).head(1)
# list_of_predictions


In [41]:
# ingredients_input = ingredients_input.split(", ")
ingredients_input = "makaron, ryżu, ogórek, jogurt, grecki, pomidor"
find_missing_ingredients(ingredients_input)
# ingredients_input = get_ingredients_alternatives(ingredients_input, similarity_threshold=0.8)
ingredients_combinations_test = combine_ingr_and_alt_into_combinations(ingredients_input, similarity_threshold=0.8)
ingredients_combinations_test = [", ".join(i) for i in ingredients_combinations_test]
get_ingredients_alternatives(ingredients_input, similarity_threshold=0.8)

[]


{'makaron': ['makaronik'],
 'ryżu': [],
 'ogórek': ['pomidor'],
 'jogurt': [],
 'grecki': [],
 'pomidor': ['pomidorek', 'ogórek']}

In [42]:
# this algo is predicting recipies based on the list of ingredients combination
# list of strings, comma separated
predi = pipe.predict_proba(ingredients_combinations_test)
result_alt={}
combination=[]
probability=[]
result_alt=[]
recipe=[]

for idx,p in enumerate(predi):
    list_of_predictions = list(zip(list(pipe.classes_), predi[idx,:]))
    list_of_predictions = pd.DataFrame(list_of_predictions)
    list_of_predictions = list_of_predictions.sort_values(by=[1], ascending=False).head(1).values.tolist()
    combination.append(ingredients_combinations_test[idx])
    recipe.append(list_of_predictions[0][0])
    probability.append(list_of_predictions[0][1])
    result_alt = {"index": combination, 'recipe':recipe, 'probability':probability}
# list_of_predictions = list(zip(list(pipe.classes_), predi[0,:]))
list_of_proposed_recipes = pd.DataFrame.from_dict(result_alt)
list_of_proposed_recipes.sort_values(by=['probability'], ascending=False)
list_of_proposed_recipes = list_of_proposed_recipes['recipe'].drop_duplicates().head(5).tolist()

alternatives = get_ingredients_alternatives(ingredients_input, similarity_threshold=0.8)
from pprint import pprint
print('Alternatives:')
pprint(alternatives)

for i in list_of_proposed_recipes:
    recipe_name = df_working.loc[df_working['link'] == i]['recipe_name'].drop_duplicates().tolist()[0]
    link = df_working.loc[df_working['link'] == i]['link'].drop_duplicates().tolist()[0]
    meal_type_detailed = df_working.loc[df_working['link'] == i]['meal_type_detailed'].drop_duplicates().tolist()[0]
    raw_ingredients = df_working.loc[df_working['link'] == i]['raw_ingredients'].drop_duplicates().tolist()[0]
    instructions = df_working.loc[df_working['link'] == i]['instructions'].drop_duplicates().tolist()[0]
    
    print('-------------------\n')
    print(recipe_name)
    print(f'Meal category: {meal_type_detailed}')
    print(f'See more on: https://kuchnialidla.pl{link}\n')
    print(f'Ingredients list: {raw_ingredients}\n')
    print('Instructions:')
    print(instructions)

Alternatives:
{'grecki': [],
 'jogurt': [],
 'makaron': ['makaronik'],
 'ogórek': ['pomidor'],
 'pomidor': ['pomidorek', 'ogórek'],
 'ryżu': []}
-------------------

Mrożony jogurt czekoladowy z ciastkami
Meal category: Lody
See more on: https://kuchnialidla.pl/mrozony-jogurt-czekoladowy-z-ciastkami

Ingredients list: jogurt grecki – 600 g, cukier – 90 g, cukier waniliowy – 1 opakowanie, kakao – 50 g, ciasteczka czekoladowe – 90 g, truskawki – 500 g

Instructions:
 Krok 1: Mrozimy jogurt Jogurt miksujemy z cukrem, cukrem waniliowym i kakao. Wstawiamy w płytkim naczyniu do zamrażarki na 1 godzinę. Krok 2: Dodajemy ciastka do jogurtu Ciastka siekamy grubo. Zmrożony jogurt mieszamy, dodajemy ciastka. Wstawiamy na kolejne 2 godziny do zamrażarki, mieszając co 30 minut ostrożnie widelcem, aby rozbić kryształki lodu. Krok 3: Przygotowujemy truskawki Truskawki oczyszczamy, kroimy na małe kawałki i układamy w salaterkach. Nakładamy na nie zamrożony jogurt czekoladowy i podajemy. 
-------------

In [59]:
%%time
X_train, X_test, y_train, y_test = train_test_split(df_working['lematized_ingredients_nouns'], 
                                                    df_working['link'], 
                                                    test_size=0.1, 
                                                    random_state=42)


pipe_norm = Pipeline([
                        ("vectorizer", CountVectorizer(analyzer='word', tokenizer=lambda x: x.split(", "))),
#                         ("scaler", StandardScaler(with_mean=False)),
                        ("regresja", LogisticRegression(solver="liblinear"))])
pipe_norm.fit(df_working['lematized_ingredients'], df_working['link'])

accuracy_score(pipe_norm.predict(X_test), y_test)

# Model need to be run, because can't pickle lambda funciton
# https://stackoverflow.com/questions/25348532/can-python-pickle-lambda-functions/25353243#25353243

CPU times: user 28.4 s, sys: 24 ms, total: 28.4 s
Wall time: 28.4 s


1.0

In [ ]:
# ingredients_input = ingredients_input.split(", ")
ingredients_input = "makaron, ryż, ogórek, jogurt, pomidor, mąka, olej, oliwa, sól, pieprz, zioło, bazylia, oregano, kapary, kakao"
find_missing_ingredients(ingredients_input)
# ingredients_input = get_ingredients_alternatives(ingredients_input, similarity_threshold=0.8)
ingredients_combinations_test = combine_ingr_and_alt_into_combinations(ingredients_input, similarity_threshold=0.75)
ingredients_combinations_test = [", ".join(i) for i in ingredients_combinations_test]
get_ingredients_alternatives(ingredients_input, similarity_threshold=0.8)

[]


In [61]:
# this algo is predicting recipies based on the list of ingredients combination
# list of strings, comma separated
predi2 = pipe_norm.predict_proba(ingredients_combinations_test)
result_alt={}
combination=[]
probability=[]
result_alt=[]
recipe=[]

for idx,p in enumerate(predi):
    list_of_predictions = list(zip(list(pipe_norm.classes_), predi2[idx,:]))
    list_of_predictions = pd.DataFrame(list_of_predictions)
    list_of_predictions = list_of_predictions.sort_values(by=[1], ascending=False).head(1).values.tolist()
    combination.append(ingredients_combinations_test[idx])
    recipe.append(list_of_predictions[0][0])
    probability.append(list_of_predictions[0][1])
    result_alt = {"index": combination, 'recipe':recipe, 'probability':probability}
# list_of_predictions = list(zip(list(pipe.classes_), predi[0,:]))
list_of_proposed_recipes = pd.DataFrame.from_dict(result_alt)
list_of_proposed_recipes.sort_values(by=['probability'], ascending=False)
list_of_proposed_recipes = list_of_proposed_recipes['recipe'].drop_duplicates().head(5).tolist()
alternatives = get_ingredients_alternatives(ingredients_input, similarity_threshold=0.8)

from pprint import pprint
print('Alternatives:')
pprint(alternatives)
for i in list_of_proposed_recipes:
    recipe_name = df_working.loc[df_working['link'] == i]['recipe_name'].drop_duplicates().tolist()[0]
    link = df_working.loc[df_working['link'] == i]['link'].drop_duplicates().tolist()[0]
    meal_type_detailed = df_working.loc[df_working['link'] == i]['meal_type_detailed'].drop_duplicates().tolist()[0]
    raw_ingredients = df_working.loc[df_working['link'] == i]['raw_ingredients'].drop_duplicates().tolist()[0]
    instructions = df_working.loc[df_working['link'] == i]['instructions'].drop_duplicates().tolist()[0]
    
    print('-------------------\n')
    print(recipe_name)
    print(f'Meal category: {meal_type_detailed}')
    print(f'See more on: https://kuchnialidla.pl{link}\n')
    print(f'Ingredients list: {raw_ingredients}\n')
    print('Instructions:')
    print(instructions)
    

Alternatives:
{'grecki': [],
 'jogurt': [],
 'makaron': ['makaronik'],
 'ogórek': ['pomidor'],
 'pomidor': ['pomidorek', 'ogórek'],
 'ryż': []}
-------------------

Słodki dżem z pomidor&#243;w
Meal category: Dżemy
See more on: https://kuchnialidla.pl/slodki-dzem-z-pomidorow

Ingredients list: 4 duże pomidory, 1 szklanka cukru, 1 łyżka soku z cytryny

Instructions:
 Przygotowanie W dużym garnku gotujemy wodę. Pomidory myjemy i nacinamy sk&#243;rkę na krzyż. Wkładamy je do wrzącej wody na minutę. Wyjmujemy, odsączamy i odstawiamy do ostygnięcia. Usuwamy z pomidor&#243;w sk&#243;rkę. Ściskamy pomidory, pozbawiając je części pestek i soku. Resztę drobno siekamy. Pomidory przekładamy do garnka. Gotujemy 15 minut, mieszając od czasu do czasu, aż rozpadną się i część płynu odparuje. Dodajemy cukier, mieszamy. Rozgotowane pomidory rozcieramy za pomocą blendera, a następnie gotujemy na małym ogniu przez kolejnych 30 minut, sporadycznie mieszając. Pod koniec gotowania dodajemy sok z cytryny. Dż

# Tuning

In [23]:
# https://www.mikulskibartosz.name/how-to-split-a-list-inside-a-dataframe-cell-into-rows-in-pandas/
missing_ingredient_list = []
meal_type_list = []
link_list = []
rest_of_ingredients_list = []
meal_type_detailed_list = []
raw_ingredients_list = []
lematized_ingredients_nouns_list = []

for _, row in df_working.iterrows():
    meal_type = row.meal_type
    link = row.link
    meal_type_detailed = row.meal_type_detailed
    raw_ingredients = row.ingredients
    lematized_ingredients_nouns = row.lematized_ingredients_nouns
    for ingredient in row.lematized_ingredients_nouns.split(", "):
        meal_type_list.append(meal_type)
        missing_ingredient_list.append(ingredient)
        link_list.append(link)
        rest_of_ingredients_list.append(", ".join([x for x in row.lematized_ingredients_nouns.split(", ") \
                                                   if x!=ingredient]))
        meal_type_detailed_list.append(meal_type_detailed)
        raw_ingredients_list.append(raw_ingredients)
        lematized_ingredients_nouns_list.append(lematized_ingredients_nouns)

df_missing_ingr_nouns_clf = pd.DataFrame({
    "link": link_list,
    "lematized_ingredients_nouns":lematized_ingredients_nouns_list,
    "missing_ingredient": missing_ingredient_list,
    "meal_type_list": meal_type_list,
    "rest_of_ingredients": rest_of_ingredients_list,
    "meal_type_detailed": meal_type_detailed_list,
    "raw_ingredients_list":raw_ingredients_list,
})

df_missing_ingr_nouns_clf

,link,lematized_ingredients_nouns,missing_ingredient,meal_type_list,rest_of_ingredients,meal_type_detailed,raw_ingredients_list
0,/ravioli-z-cukinia-i-ricotta,"sól, pieprz, ser, bazylia, masło, ricotta, mąk...",sól,glowne,"pieprz, ser, bazylia, masło, ricotta, mąka, ch...",Bez mięsa,"cukinia – 1 szt., oliwa z oliwek – 1 łyżka, cz..."
1,/ravioli-z-cukinia-i-ricotta,"sól, pieprz, ser, bazylia, masło, ricotta, mąk...",pieprz,glowne,"sól, ser, bazylia, masło, ricotta, mąka, chili...",Bez mięsa,"cukinia – 1 szt., oliwa z oliwek – 1 łyżka, cz..."
2,/ravioli-z-cukinia-i-ricotta,"sól, pieprz, ser, bazylia, masło, ricotta, mąk...",ser,glowne,"sól, pieprz, bazylia, masło, ricotta, mąka, ch...",Bez mięsa,"cukinia – 1 szt., oliwa z oliwek – 1 łyżka, cz..."
3,/ravioli-z-cukinia-i-ricotta,"sól, pieprz, ser, bazylia, masło, ricotta, mąk...",bazylia,glowne,"sól, pieprz, ser, masło, ricotta, mąka, chili,...",Bez mięsa,"cukinia – 1 szt., oliwa z oliwek – 1 łyżka, cz..."
4,/ravioli-z-cukinia-i-ricotta,"sól, pieprz, ser, bazylia, masło, ricotta, mąk...",masło,glowne,"sól, pieprz, ser, bazylia, ricotta, mąka, chil...",Bez mięsa,"cukinia – 1 szt., oliwa z oliwek – 1 łyżka, cz..."
...,...,...,...,...,...,...,...
42423,/mus-czekoladowy,"carre, fin, czekolada, pilos, śmietanka",carre,desery,"fin, czekolada, pilos, śmietanka",Desery czekoladowe,"śmietanka Pilos 30%, mocno schłodzona: 300 ml,..."
42424,/mus-czekoladowy,"carre, fin, czekolada, pilos, śmietanka",fin,desery,"carre, czekolada, pilos, śmietanka",Desery czekoladowe,"śmietanka Pilos 30%, mocno schłodzona: 300 ml,..."
42425,/mus-czekoladowy,"carre, fin, czekolada, pilos, śmietanka",czekolada,desery,"carre, fin, pilos, śmietanka",Desery czekoladowe,"śmietanka Pilos 30%, mocno schłodzona: 300 ml,..."
42426,/mus-czekoladowy,"carre, fin, czekolada, pilos, śmietanka",pilos,desery,"carre, fin, czekolada, śmietanka",Desery czekoladowe,"śmietanka Pilos 30%, mocno schłodzona: 300 ml,..."


In [26]:
%%time
X_train, X_test, y_train, y_test = train_test_split(df_missing_ingr_nouns_clf['lematized_ingredients_nouns'], 
                                                    df_missing_ingr_nouns_clf['link'], 
                                                    test_size=0.1, 
                                                    random_state=42)


pipe_norm = Pipeline([
                        ("vectorizer", CountVectorizer(analyzer='word', tokenizer=lambda x: x.split(", "))),
#                         ("scaler", StandardScaler(with_mean=False)),
                        ("regresja", LogisticRegression(solver="liblinear"))])
pipe_norm.fit(df_working['lematized_ingredients'], df_working['link'])

accuracy_score(pipe_norm.predict(X_test), y_test)

# Model need to be run, because can't pickle lambda funciton
# https://stackoverflow.com/questions/25348532/can-python-pickle-lambda-functions/25353243#25353243

CPU times: user 29.9 s, sys: 100 ms, total: 30 s
Wall time: 30 s


0.9992929530992223

In [ ]:
# ingredients_input = ingredients_input.split(", ")
ingredients_input = "makaron, ryż, ogórek, jogurt, pomidor, mąka, olej, oliwa, sól, pieprz, jajko, oregano, kapary, kakao"
find_missing_ingredients(ingredients_input)
# ingredients_input = get_ingredients_alternatives(ingredients_input, similarity_threshold=0.8)
ingredients_combinations_test = combine_ingr_and_alt_into_combinations(ingredients_input, similarity_threshold=0.70)
ingredients_combinations_test = [", ".join(i) for i in ingredients_combinations_test]
get_ingredients_alternatives(ingredients_input, similarity_threshold=0.70)

[]


In [1]:
# this algo is predicting recipies based on the list of ingredients combination
# list of strings, comma separated
predi2 = pipe_norm.predict_proba(ingredients_combinations_test)
result_alt={}
combination=[]
probability=[]
result_alt=[]
recipe=[]

for idx,p in enumerate(predi2):
    list_of_predictions = list(zip(list(pipe_norm.classes_), predi2[idx,:]))
    list_of_predictions = pd.DataFrame(list_of_predictions)
    list_of_predictions = list_of_predictions.sort_values(by=[1], ascending=False).head(1).values.tolist()
    combination.append(ingredients_combinations_test[idx])
    recipe.append(list_of_predictions[0][0])
    probability.append(list_of_predictions[0][1])
    result_alt = {"index": combination, 'recipe':recipe, 'probability':probability}
# list_of_predictions = list(zip(list(pipe.classes_), predi[0,:]))
list_of_proposed_recipes = pd.DataFrame.from_dict(result_alt)
list_of_proposed_recipes.sort_values(by=['probability'], ascending=False)
list_of_proposed_recipes = list_of_proposed_recipes['recipe'].drop_duplicates().head(1).tolist()
alternatives = get_ingredients_alternatives(ingredients_input, similarity_threshold=0.8)

from pprint import pprint
print('Alternatives:')
pprint(alternatives)
for i in list_of_proposed_recipes:
    recipe_name = df_working.loc[df_working['link'] == i]['recipe_name'].drop_duplicates().tolist()[0]
    link = df_working.loc[df_working['link'] == i]['link'].drop_duplicates().tolist()[0]
    meal_type_detailed = df_working.loc[df_working['link'] == i]['meal_type_detailed'].drop_duplicates().tolist()[0]
    raw_ingredients = df_working.loc[df_working['link'] == i]['raw_ingredients'].drop_duplicates().tolist()[0]
    instructions = df_working.loc[df_working['link'] == i]['instructions'].drop_duplicates().tolist()[0]
    
    print('-------------------\n')
    print(recipe_name)
    print(f'Meal category: {meal_type_detailed}')
    print(f'See more on: https://kuchnialidla.pl{link}\n')
    print(f'Ingredients list: {raw_ingredients}\n')
    print('Instructions:')
    print(instructions)
    

NameError: name 'pipe_norm' is not defined